# Red blood cell classifier : practical application of a convNet

With this example, you will build your own convNet in order to perform a specific task : classifying images of RBC between good and bad quality.

Import the keras libraries to build and train the network

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
import matplotlib.pyplot as plt

Define the folder where the RBC data are saved and described the folders where the different classes are kept.Again, three sets of data are available : training, testing and validation.


In [ ]:
base_dir = './RBC_database'

# Create a folder for the training, validation and testing data. 
# For each of them, two folders are created, one for the good RBC
# images, and the other for the bad ones.
# --------------------------------------

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_goodRBC_dir = os.path.join(train_dir,'goodRBC')
train_badRBC_dir = os.path.join(train_dir,'badRBC')

validation_goodRBC_dir = os.path.join(validation_dir,'goodRBC')
validation_badRBC_dir = os.path.join(validation_dir,'badRBC')

test_goodRBC_dir = os.path.join(test_dir,'goodRBC')
test_badRBC_dir = os.path.join(test_dir,'badRBC')

Preprocess the images. The idea here is to create the mini-bacth of images using an
image generator. Also the images are normalized in such way the maximum intensity is
equal to 1. 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(50,50),
        batch_size = 64,
        class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(50,50),
        batch_size = 64,
        class_mode = 'binary')

Define the model and the compilation options. 


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50,50,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr=1e-4), 
            loss='binary_crossentropy',
            metrics=['accuracy'])

model.summary()

Define the model and the compilation options. 

In [ ]:
history = model.fit_generator(train_generator,
                    steps_per_epoch = 100,
                    epochs = 1,
                    validation_data = validation_generator,
                    validation_steps = 50)

Display the loss function during the training


In [ ]:
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

n = len(loss_values)
epochs = range(1, n+1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

The accuracy of the model is tested using the testing set of data.

In [ ]:
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

n = len(acc_values)
epochs = range(1, n+1)

plt.plot(epochs, acc_values, 'bo', label='Training acccuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()

plt.show()

Finally test the accuracy of the model directly on the testing set of images

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(50,50),
        batch_size = 20,
        class_mode = 'binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps = 50)
print('test_acc : ', test_acc)
print('test_loss : ', test_loss)